# Automatic iOS photo and video curation and archiving from its iTunes backup files

## Asset types


| Kind | Kind subtype | PLAYBACK STYLE | PLAYBACK VARIATION | UNIFORM TYPE IDENTIFIER    | Description              |
|------|--------------|----------------|--------------------|----------------------------|--------------------------|
| 0    | 0            | 1              | 0                  | public.jpeg or public.heic | Photo                    |
| 0    | 0            | 1              | 0                  | public.jpeg or public.heic | Portrait                 |
| 0    | 10           | 1              | 0                  | public.png                 | Screenshot               |
| 0    | 2            | 3              | 0                  | public.jpeg or public.heic | Live Photo               |
| 0    | 2            | 5              | 1                  | public.jpeg or public.heic | Live Photo loop          |
| 0    | 2            | 5              | 2                  | public.jpeg or public.heic | Live Photo bounce        |
| 0    | 2            | 3              | 3                  | public.jpeg or public.heic | Live Photo long exposure |
| 1    | 0            | 4              | 0                  | com.apple.quicktime-movie  | Video                    |
| 1    | 101          | 4              | 0                  | com.apple.quicktime-movie  | Slowmotion               |
| 1    | 102          | 4              | 0                  | com.apple.quicktime-movie  | Timelapse                |
| 1    | 103          | 4              | 0                  | public.mpeg-4              | Screen recording         |

Following query on iOS 14’s `CameraRollDomain~Media/PhotoData/Photos.sqlite` will provide a good amount of data about assets:

```sql
 select
    ZASSET.Z_PK as Asset_PK,
    ZASSET.ZDIRECTORY as dcim_folder,
    ZASSET.ZFILENAME as file,
    ZASSETDESCRIPTION.ZLONGDESCRIPTION as caption,
    ZASSET.ZTRASHEDSTATE as trashed,
    ZASSET.ZHASADJUSTMENTS as edited,                
    case
        when ZASSET.ZKIND=0 then case -- images
            when ZASSET.ZKINDSUBTYPE=2 then '0>'||ZASSET.ZKINDSUBTYPE||'>'||ZASSET.ZPLAYBACKSTYLE||'>'||ZASSET.ZPLAYBACKVARIATION
            when ZASSET.ZPLAYBACKSTYLE=2 then '0>'||ZASSET.ZKINDSUBTYPE||'>'||ZASSET.ZPLAYBACKSTYLE
            else '0>'||ZASSET.ZKINDSUBTYPE
        end
        when ZASSET.ZKIND=1 then ZASSET.ZKIND||'>'||ZASSET.ZKINDSUBTYPE --videos
    end as kind_encoded,
    case 
        when ZASSET.ZKIND=0 then case -- images
            when ZASSET.ZKINDSUBTYPE=0 then case
                when ZASSET.ZPLAYBACKSTYLE=2 then 'animated GIF'
                else 'image' -- check ZCREATORBUNDLEID if its a PNG
            end
            when ZASSET.ZKINDSUBTYPE=10 then 'screenshot' -- created by this device
            when ZASSET.ZKINDSUBTYPE=2 then case -- live photos
                when ZASSET.ZPLAYBACKSTYLE=3 then case
                    when ZASSET.ZPLAYBACKVARIATION=0 then 'live photo'
                    when ZASSET.ZPLAYBACKVARIATION=3 then 'long exposure from live photo'
                end
                when ZASSET.ZPLAYBACKSTYLE=5 then case
                    when ZASSET.ZPLAYBACKVARIATION=1 then 'live photo, loop effect'
                    when ZASSET.ZPLAYBACKVARIATION=2 then 'live photo, bounce effect'
                end
            end
        end
        when ZASSET.ZKIND=1 then case -- videos
            when ZASSET.ZKINDSUBTYPE=0 then 'video'
            when ZASSET.ZKINDSUBTYPE=101 then 'slow motion video'
            when ZASSET.ZKINDSUBTYPE=102 then 'timelapse video'
            when ZASSET.ZKINDSUBTYPE=103 then 'screencast'
        end
    end as kind_description,
    ZASSET.ZFAVORITE as favorited,
    ZASSET.ZWIDTH as width,
    ZASSET.ZHEIGHT as height,
    ZASSET.ZDURATION as video_duration,
    ZASSET.ZUUID as uuid,
    ZMOMENT.ZTITLE as moment_title,
    ZMOMENT.ZSUBTITLE as moment_subtitle,
    ZASSET.ZDATECREATED as creation_timestamp,
    datetime(ZASSET.ZDATECREATED+strftime('%s','2001-01-01'),'unixepoch') as utc_time,
    ZADDITIONALASSETATTRIBUTES.ZINFERREDTIMEZONEOFFSET as tz_offset,
    datetime(ZASSET.ZDATECREATED+ZADDITIONALASSETATTRIBUTES.ZINFERREDTIMEZONEOFFSET+strftime('%s','2001-01-01'),'unixepoch') as asset_local_time,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING as exif_timestamp,
    ZADDITIONALASSETATTRIBUTES.ZREVERSELOCATIONDATA as location_data,
    facecount.facecount,
    named_facecount.named_facecount
from
    ZASSET

    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZASSET.Z_PK

    left outer join ZASSETDESCRIPTION
        on ZASSETDESCRIPTION.ZASSETATTRIBUTES=ZADDITIONALASSETATTRIBUTES.Z_PK

    left outer join ZMOMENT
        on ZMOMENT.Z_PK=ZASSET.ZMOMENT

    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZASSET.Z_PK

    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZASSET.Z_PK

where
    ZASSET.ZDIRECTORY like 'DCIM%'
    
order by ZASSET.ZDATECREATED desc;
```


```sql
attach database '/Users/avi/Notebooks/iOSbackup/playground/Sample--Manifest.db' as manifest;

select
    ZASSET.Z_PK as Asset_PK,
    ZASSET.ZDIRECTORY as dcim_folder,
    ZASSET.ZFILENAME as file,
    replace(ZASSET.ZFILENAME,
            '.' || replace(ZASSET.ZFILENAME,rtrim(ZASSET.ZFILENAME, replace(ZASSET.ZFILENAME, '.', '') ), ''),
            '') as file_no_extension
    
from
    ZASSET

where
    ZASSET.ZDIRECTORY like 'DCIM%'
    and files.domain='CameraRollDomain'
    and files.relativePath like 'Media/PhotoData/Mutations/'||dcim_folder||'/'||file_no_extension||'/Adjustments/FullSizeRender%'

order by ZASSET.ZDATECREATED desc;
```
## List all masters and mutations related to Photo Assets
```sql
select
    ZASSET.Z_PK as Asset_PK,
    ZASSET.ZDIRECTORY as dcim_folder,
    ZASSET.ZFILENAME as asset_file,
    replace(
        ZASSET.ZFILENAME,
        '.' || replace(ZASSET.ZFILENAME,rtrim(ZASSET.ZFILENAME, replace(ZASSET.ZFILENAME, '.', '') ), ''),
        ''
    ) as asset_file_no_extension,
    files.relativePath
from
    ZASSET,
    manifest.Files as files

where
    ZASSET.ZDIRECTORY like 'DCIM%'
    and files.domain='CameraRollDomain'
    and (
        files.relativePath like 'Media/PhotoData/Mutations/'||dcim_folder||'/'||asset_file_no_extension||'/Adjustments/FullSizeRender%'
        or
        files.relativePath = 'Media/'||dcim_folder||'/'||asset_file
    )

order by ZASSET.ZDATECREATED desc;
```

In [ ]:
import sys
# sys.path.insert(0,"..") # Adds higher directory to python modules path.

import pprint
import logging
import datetime
import math
from PhotoCuration import *

# logging.basicConfig(level=logging.DEBUG)

# Fotos da Helena

In [ ]:
engine=PhotoCuration(
    udid='00008020-000815A42280402E',
    backupDerivedKey='8d506e9c5150641f457d10ae14481ff6b9bb88cd4bbeb2eb5a2e78844b1a931b',
    backupRoot='/media/sf_1TB_Preto/Apple/MobileSync/Backup'
)

In [ ]:
%%time
engine.curatedArchiving(
    start=datetime.datetime(2020,1,21,12),
    end=datetime.datetime(2020,2,6),
    author='Helena Leite',
    deviceOwner='Helena Leite',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
    target='/media/sf_1TB_Preto/helena'
)

engine.close()
del engine

# Fotos da Tati

In [ ]:
engine=PhotoCuration(
    udid='00008020-0002783626F1002E',
    backupDerivedKey='3cce9c0d621999a195978b69e285aaeff457f94e786f431665e52fd3000a0592',
    backupRoot='/media/sf_1TB_Preto/Apple/MobileSync/Backup'
)

In [ ]:
%%time
engine.curatedArchiving(
    start=datetime.datetime(2020,1,21,12),
    end=datetime.datetime(2020,2,6),
    author='Tatiana Cymbalista',
    deviceOwner='Tatiana Cymbalista',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
    target='/media/sf_1TB_Preto/tatiana'
)

engine.close()
del engine

# Fotos do Avi

In [ ]:
engine=PhotoCuration(
    udid='00008030-000E44981482802E',
    backupDerivedKey='f693b40d0c6c7c03c8820eef4b17744c522ce052e5db0fbe9d7ccdc96c8638ff',
    backupRoot='/media/sf_aviram/Library/Application Support/MobileSync/Backup/'
)

In [ ]:
%%time
engine.curatedArchiving(
    start=datetime.datetime(2020,1,22,8,27,40),
    end=datetime.datetime(2020,2,5,22,38),
    author='Avi Alkalay',
    deviceOwner='Avi Alkalay',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
    target='/media/sf_1TB_Preto/mobileavi/2020-01 Orlando'
)

In [ ]:
%%time
engine.curatedArchiving(
    start=datetime.datetime(2020,12,14),
#     end=datetime.datetime(2020,5,2,12,40),
    author='Avi Alkalay',
    deviceOwner='Avi Alkalay',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
    target='/media/sf_1TB_Preto/mobileavi/'
)

In [ ]:
engine.assets.columns

In [ ]:
with pd.ExcelWriter(f'mobileavi-assets.xlsx') as writer:  
    engine.assets[engine.assets['trashed']==0].sort_values(by='utc_time')[['asset_local_time','location_suggested_name','infered_asset_caption']].to_excel(writer, sheet_name='Assets')

In [ ]:
engine.close()
del engine

# Fotos de teste

In [ ]:
%%time

engine=PhotoCuration(
    udid='00008030-000E44981482802E',
    backupDerivedKey='f693b40d0c6c7c03c8820eef4b17744c522ce052e5db0fbe9d7ccdc96c8638ff',
    backupRoot='/media/sf_aviram/Library/Application Support/MobileSync/Backup/'
)

engine.curatedArchiving(
#    start=datetime.datetime(2020,10,25), end=datetime.datetime(2020,10,26,9,19),
    start=datetime.datetime(2021,1,1,0,4), end=datetime.datetime(2021,1,1,0,5),

    author='Avi Alkalay',
    deviceOwner='Avi Alkalay',
#     originals=True,
#     extractTypes=['0>0', '0>0>2', '0>10', '0>0>1>2'],
#     extractTypes=['1>0', '1>101', '1>102', '1>103'],
#     extractTypes=['1>101'],
    target='aa-uu-2'
)

In [ ]:
engine.assetCurrent

## All data about photos

```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    ZGENERICASSET.ZDIRECTORY,
    ZGENERICASSET.ZFILENAME,
    ZGENERICASSET.ZTRASHEDSTATE,
    ZGENERICASSET.ZFAVORITE,
    ZGENERICASSET.ZWIDTH,
    ZGENERICASSET.ZHEIGHT,
    ZMOMENT.ZTITLE as Moment_Title,
    ZMOMENT.ZSUBTITLE as Moment_Subtitle,
    datetime(ZGENERICASSET.ZDATECREATED+strftime('%s','2001-01-01','utc'),'unixepoch') as taken,
    ZADDITIONALASSETATTRIBUTES.ZTIMEZONEOFFSET,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING,
    facecount.facecount,
    named_facecount.named_facecount,
    people.Detected_PK,
    people.Person_PK,
    people.short_name,
    people.full_name,
    people.adjustment_ver,
    people.width,
    people.height,
    people.LEFTEYE_pixx,
    people.LEFTEYE_pixy,
    people.RIGHTEYE_pixx,
    people.RIGHTEYE_pixy,
    people.MOUTH_pixx,
    people.MOUTH_pixy,
    people.CENTER_pixx,
    people.CENTER_pixy,
    people.face_size,
    people.person_uri,
    people.person_uuid
from
    ZGENERICASSET
    
    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZGENERICASSET.Z_PK
        
    left outer join ZMOMENT
        on ZMOMENT.Z_PK= ZGENERICASSET.ZMOMENT
        
    left outer join (
        select 
            ZDETECTEDFACE.Z_PK as Detected_PK,
            ZPERSON.Z_PK as Person_PK,
            ZPERSON.ZDISPLAYNAME as short_name,
            ZPERSON.ZFULLNAME as full_name,
            ZDETECTEDFACE.ZADJUSTMENTVERSION as adjustment_ver,
            ZDETECTEDFACE.ZSOURCEWIDTH as width,
            ZDETECTEDFACE.ZSOURCEHEIGHT as height,
            cast(round(ZDETECTEDFACE.ZLEFTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) LEFTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZLEFTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) LEFTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) RIGHTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) RIGHTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZMOUTHX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) MOUTH_pixx,
            cast(round(ZDETECTEDFACE.ZMOUTHY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) MOUTH_pixy,
            cast(round(ZDETECTEDFACE.ZCENTERX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) CENTER_pixx,
            cast(round(ZDETECTEDFACE.ZCENTERY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) CENTER_pixy,
            ZDETECTEDFACE.ZSIZE as face_size, -- face_size×√(width²+height²) = diameter of circle surrounding face
            ZPERSON.ZPERSONURI as person_uri,
            ZPERSON.ZPERSONUUID as person_uuid,
            ZDETECTEDFACE.ZASSET as asset
        from
            ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
    ) as people
        on people.asset=ZGENERICASSET.Z_PK
    
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZGENERICASSET.Z_PK
    
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZGENERICASSET.Z_PK
    
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5

order by
        taken, Detected_PK
```



## List of Assets


```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    ZGENERICASSET.ZDIRECTORY,
    ZGENERICASSET.ZFILENAME,
    ZGENERICASSET.ZTRASHEDSTATE,
    ZGENERICASSET.ZFAVORITE,
    ZGENERICASSET.ZWIDTH,
    ZGENERICASSET.ZHEIGHT,
    ZMOMENT.ZTITLE as Moment_Title,
    ZMOMENT.ZSUBTITLE as Moment_Subtitle,
    datetime(ZGENERICASSET.ZDATECREATED+strftime('%s','2001-01-01','utc'),'unixepoch') as utc_time,
    ZADDITIONALASSETATTRIBUTES.ZTIMEZONEOFFSET,
    ZADDITIONALASSETATTRIBUTES.ZEXIFTIMESTAMPSTRING,
    facecount.facecount,
    named_facecount.named_facecount
from
    ZGENERICASSET
    
    left outer join ZADDITIONALASSETATTRIBUTES
        on ZADDITIONALASSETATTRIBUTES.ZASSET=ZGENERICASSET.Z_PK
        
    left outer join ZMOMENT
        on ZMOMENT.Z_PK= ZGENERICASSET.ZMOMENT
        
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as facecount
        from ZDETECTEDFACE
        group by ZASSET
    ) as facecount
        on facecount.asset=ZGENERICASSET.Z_PK
    
    left outer join (
        select
            ZASSET as asset,
            count(ZCENTERX) as named_facecount
        from ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
        group by ZASSET
    ) as named_facecount
        on named_facecount.asset=ZGENERICASSET.Z_PK
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5
```



## Faces for Asset

```sql
select
    ZGENERICASSET.Z_PK as Asset_PK,
    people.Detected_PK,
    people.Person_PK,
    people.short_name,
    people.full_name,
    people.adjustment_ver,
    people.width,
    people.height,
    people.LEFTEYE_pixx,
    people.LEFTEYE_pixy,
    people.RIGHTEYE_pixx,
    people.RIGHTEYE_pixy,
    people.MOUTH_pixx,
    people.MOUTH_pixy,
    people.CENTER_pixx,
    people.CENTER_pixy,
    people.face_size,
    people.person_uri,
    people.person_uuid
from
    (
        select 
            ZDETECTEDFACE.Z_PK as Detected_PK,
            ZPERSON.Z_PK as Person_PK,
            ZPERSON.ZDISPLAYNAME as short_name,
            ZPERSON.ZFULLNAME as full_name,
            ZDETECTEDFACE.ZADJUSTMENTVERSION as adjustment_ver,
            ZDETECTEDFACE.ZSOURCEWIDTH as width,
            ZDETECTEDFACE.ZSOURCEHEIGHT as height,
            cast(round(ZDETECTEDFACE.ZLEFTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) LEFTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZLEFTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) LEFTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) RIGHTEYE_pixx,
            cast(round(ZDETECTEDFACE.ZRIGHTEYEY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) RIGHTEYE_pixy,
            cast(round(ZDETECTEDFACE.ZMOUTHX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) MOUTH_pixx,
            cast(round(ZDETECTEDFACE.ZMOUTHY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) MOUTH_pixy,
            cast(round(ZDETECTEDFACE.ZCENTERX * ZDETECTEDFACE.ZSOURCEWIDTH,0) as integer) CENTER_pixx,
            cast(round(ZDETECTEDFACE.ZCENTERY * ZDETECTEDFACE.ZSOURCEHEIGHT,0) as integer) CENTER_pixy,
            ZDETECTEDFACE.ZSIZE as face_size, -- face_size×√(width²+height²) = diameter of circle surrounding face
            ZPERSON.ZPERSONURI as person_uri,
            ZPERSON.ZPERSONUUID as person_uuid,
            ZDETECTEDFACE.ZASSET as asset
        from
            ZDETECTEDFACE, ZPERSON
        where
            ZDETECTEDFACE.ZPERSON=ZPERSON.Z_PK and
            ZPERSON.ZDISPLAYNAME!=''
    ) as people
    
    left outer join ZGENERICASSET
        on ZGENERICASSET.Z_PK=people.asset
    
where
    ZGENERICASSET.ZDIRECTORY like 'DCIM%'
--  and Person_PK=5

order by
        Asset_PK desc
```


